# Supervised Learning - Foundations: ReCell 

***Marks: 60***

## Context

Buying and selling used phones and tablets used to be something that happened on a handful of online marketplace sites. But the used and refurbished device market has grown considerably over the past decade, and a new IDC (International Data Corporation) forecast predicts that the used phone market would be worth \\$52.7bn by 2023 with a compound annual growth rate (CAGR) of 13.6% from 2018 to 2023. This growth can be attributed to an uptick in demand for used phones and tablets that offer considerable savings compared with new models.

Refurbished and used devices continue to provide cost-effective alternatives to both consumers and businesses that are looking to save money when purchasing one. There are plenty of other benefits associated with the used device market. Used and refurbished devices can be sold with warranties and can also be insured with proof of purchase. Third-party vendors/platforms, such as Verizon, Amazon, etc., provide attractive offers to customers for refurbished devices. Maximizing the longevity of devices through second-hand trade also reduces their environmental impact and helps in recycling and reducing waste. The impact of the COVID-19 outbreak may further boost this segment as consumers cut back on discretionary spending and buy phones and tablets only for immediate needs.

 
## Objective

The rising potential of this comparatively under-the-radar market fuels the need for an ML-based solution to develop a dynamic pricing strategy for used and refurbished devices. ReCell, a startup aiming to tap the potential in this market, has hired you as a data scientist. They want you to analyze the data provided and build a linear regression model to predict the price of a used phone/tablet and identify factors that significantly influence it.

 
## Data Description
The data contains the different attributes of used/refurbished phones and tablets. The detailed data dictionary is given below.

**Data Dictionary**

- brand_name: Name of manufacturing brand
- os: OS on which the device runs
- screen_size: Size of the screen in cm
- 4g: Whether 4G is available or not
- 5g: Whether 5G is available or not
- main_camera_mp: Resolution of the rear camera in megapixels
- selfie_camera_mp: Resolution of the front camera in megapixels
- int_memory: Amount of internal memory (ROM) in GB
- ram: Amount of RAM in GB
- battery: Energy capacity of the device battery in mAh
- weight: Weight of the device in grams
- release_year: Year when the device model was released
- days_used: Number of days the used/refurbished device has been used
- new_price: Price of a new device of the same model in euros
- used_price: Price of the used/refurbished device in euros

## Importing necessary libraries and data

In [ ]:
# Basic package imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Import Data 
udd = pd.read_csv('used_device_data.csv')
# Create a copy 
df = udd.copy()

pd.set_option('display_max_columns', None)
pd.set_option('display_max_rows', 250)

## Data Overview

- Observations
- Sanity checks

In [ ]:
# Examine data shape
print(f'There are {udd.shape[0]} rows and {udd.shape[1]} columns.')

In [ ]:
# I want to a sample of 18 random rows .5% of the data
# using seed to keep the same 18 rows sample over everytime I want to see a new sample 
#Random sample of rows
np.random.seed(1)
udd.sample(18)

In [ ]:
udd.info()

In [ ]:
# checking for missing values
udd.isnull().sum().sort_values(ascending=False)

In [ ]:
udd.duplicated(keep = False).sum()

In [ ]:
udd[udd['os'] == 'Others']['brand_name'].value_counts()

## Exploratory Data Analysis (EDA)

- EDA is an important part of any project involving data.
- It is important to investigate and understand the data better before building a model with it.
- A few questions have been mentioned below which will help you approach the analysis in the right manner and generate insights from the data.
- A thorough analysis of the data, in addition to the questions mentioned below, should be done.

**Questions**:

1. What does the distribution of used device prices look like?
2. What percentage of the used device market is dominated by Android devices?
3. The amount of RAM is important for the smooth functioning of a device. How does the amount of RAM vary with the brand?
4. A large battery often increases a device's weight, making it feel uncomfortable in the hands. How does the weight vary for phones and tablets offering large batteries (more than 4500 mAh)?
5. Bigger screens are desirable for entertainment purposes as they offer a better viewing experience. How many phones and tablets are available across different brands with a screen size larger than 6 inches?
6. Budget devices nowadays offer great selfie cameras, allowing us to capture our favorite moments with loved ones. What is the distribution of budget devices offering greater than 8MP selfie cameras across brands?
7. Which attributes are highly correlated with the price of a used device?

In [ ]:
udd.describe().T

In [ ]:
plt.figure(figsize=(20,10))
sns.distplot(udd.used_price);
plt.axvline(udd.used_price.mean(),
           color='red',
           ls='--')
plt.axvline(udd.used_price.median(),
           color='green',
           ls='-')

In [ ]:
#Function to create a labeled barplot

def labeled_barplot(data, feature, perc=False, n=None):
    """ 
    Barplot with percentages at the top 
    
    data = dataframe
    feature = dataframe column
    perc = to display percentages instead of count (default = False)
    n: displays the top n category levels (default = None - displays all levels)
    """
    total = len(data[feature]) #length of the column 
    count = data[feature].nunique()
    if n is None:
        plt.figure(figsize=(count +1,5))
    else:
        plt.figure(figsize=(n +1, 5))
        
    plt.xticks(rotation=90, fontsize=12)
    ax = sns.countplot(
        data=data,
        x=feature,
        palette="Paired",
        order=data[feature].value_counts().index[:n].sort_values(),
    )
    
    for p in ax.patches:
        if perc == True:
            label = "{:.1f}%".format(
            100 * p.get_height()/ total
            ) #gives % of the category
        else:
            label = p.get_height() #gives count of each level
        x = p.get_x() + p.get_width() / 2
        y = p.get_height() 
        
        ax.annotate(
            label,
            (x,y),
            ha='center',
            va='center',
            size=12,
            xytext=(0,5),
            textcoords="offset points",
        )
        
    plt.show()
            

In [ ]:
plt.figure(figsize= (20,10))
labeled_barplot(udd, 'os', perc=True)

In [ ]:
labeled_barplot(udd, 'brand_name', perc=True, n=10)

In [ ]:
labeled_barplot(udd, '4g')

In [ ]:
labeled_barplot(udd, '5g')

In [ ]:
nog = np.where('no', '4g', '5g')
print(nog)

In [ ]:
plt.figure(figsize= (20,10))
sns.swarmplot(data=udd, x = 'ram', y='brand_name', )

In [ ]:
plt.figure(figsize= (20,10))
sns.jointplot(data =udd, x='battery', y='weight')

## Data Preprocessing

- Missing value treatment
- Feature engineering (if needed)
- Outlier detection and treatment (if needed)
- Preparing data for modeling
- Any other preprocessing steps (if needed)

In [ ]:
udd.loc[udd['brand_name']=='LG','os'] = 'Android'
udd.loc[udd['brand_name']=='Samsung', 'os'] = 'Android'
udd.loc[udd['brand_name']=='Apple', 'os'] = 'iOS'
udd.loc[udd['brand_name']=='Alcatel','os'] = 'Android'
udd.loc[udd['brand_name']=='Micromax','os'] = 'Android'

In [ ]:
udd[udd['os'] == 'Others']['brand_name'].value_counts()

## EDA

- It is a good idea to explore the data once again after manipulating it.

## Building a Linear Regression model

## Model performance evaluation

## Checking Linear Regression Assumptions

- In order to make statistical inferences from a linear regression model, it is important to ensure that the assumptions of linear regression are satisfied.

## Final Model Summary

## Actionable Insights and Recommendations

- 